In [26]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table_experiments as dt
import pandas as pd
import plotly.graph_objs as go
from dash.dependencies import Input, Output, State
import random
from jupyter_dash import JupyterDash

In [27]:
##############################################################
        #DATA MANIPULATION (MODEL)
##############################################################
df= pd.read_csv("top500_albums_clean.csv")
df['userscore'] = df['userscore'].astype(float)
df['metascore'] = df['metascore'].astype(float)
df['releasedate']=pd.to_datetime(df['releasedate'], format='%b %d, %Y')
df['year']=df["releasedate"].dt.year
df['decade']=(df["year"]//10)*10
#cleaning Genre
df['genre'] = df['genre'].str.strip()
df['genre'] = df['genre'].str.replace("/", ",")
df['genre'] = df['genre'].str.split(",")
#year trend
df_linechart= df.groupby('year')        \
    .agg({'album':'size', 'metascore':'mean', 'userscore':'mean'})        \
    .sort_values(['year'], ascending=[True]).reset_index()
df_linechart.userscore=df_linechart.userscore*10
#table
df_table= df.groupby('artist').agg({'album':'size', 'metascore':'sum', 'userscore':'sum'})
#genrebubble
df2=(df['genre'].apply(lambda x: pd.Series(x)) .stack().reset_index(level=1, drop=True).to_frame('genre').join(df[['year', 'decade', 'userscore', 'metascore']], how='left') )
df_bubble=  df2.groupby('genre')        \
    .agg({'year':'size', 'metascore':'mean', 'userscore':'mean'})             \
    .sort_values(['year'], ascending=[False]).reset_index().head(15)
df2_decade=df2.groupby(['genre', 'decade']).agg({'year':'size'}) .sort_values(['decade'], ascending=[False]).reset_index()

In [28]:
df.head(2)

,Unnamed: 0,album,artist,genre,metascore,releasedate,userscore,year,decade
0,0,Ten Freedom Summers,Wadada Leo Smith,[Jazz],99.0,2012-05-22,5.2,2012.0,2010.0
1,1,No Cities to Love,Sleater-Kinney,"[Pop, Rock, Alternative, Indie Rock, Indie Rock]",90.0,2015-01-20,8.1,2015.0,2010.0


In [29]:
df_linechart.head()

,year,album,metascore,userscore
0,1999.0,3,87.000000,87.000000
1,2000.0,9,87.888889,86.666667
2,2001.0,24,87.250000,85.583333
3,2002.0,12,87.083333,86.416667
4,2003.0,30,86.666667,86.466667


In [30]:
df_table.head()

,album,metascore,userscore
artist,,,
...And You Will Know Us by the Trail of Dead,1,85.0,8.9
24 Hour Party People,1,86.0,8.5
9th Wonder,1,87.0,7.0
A Tribe Called Quest,1,91.0,8.8
Aesop Rock,1,85.0,8.6


In [31]:
df_bubble.head()

,genre,year,metascore,userscore
0,Rock,358,86.801676,8.303631
1,Pop,207,87.009662,8.144928
2,Alternative,125,86.776000,8.342400
3,Indie Rock,113,86.707965,8.279646
4,Indie,96,86.677083,8.560417


In [32]:
df2_decade.head()

,genre,decade,year
0,Indie Folk,2010.0,9
1,Math Rock,2010.0,1
2,Metalcore,2010.0,1
3,Microsound,2010.0,1
4,Midwest Rap,2010.0,3


In [33]:
#gererate table
def generate_table(dataframe, max_rows=10):
    '''Given dataframe, return template generated using Dash components
    '''
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))],
        style={'width': '100%', 'display': 'inline-block', 'vertical-align': 'middle'}
    )

In [34]:
# generate bar chart
def bar(results):
    gen = results["points"][0]["text"]
    figure = go.Figure(
        data=[
            go.Bar(x=df2_decade[df2_decade.genre == gen].decade, y=df2_decade[df2_decade.genre == gen].year)
        ],
        layout=go.Layout(
            title="Decade populatrity of " + gen
        )
    )
    return figure

In [35]:
# Set up Dashboard and create layout
#app = dash.Dash()
app = JupyterDash

# Bootstrap CSS.
app.css.append_css({
    "external_url": "https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css"
})
# Bootstrap Javascript.
app.scripts.append_script({
    "external_url": "https://code.jquery.com/jquery-3.2.1.slim.min.js"
})
app.scripts.append_script({
    "external_url": "https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.12.9/umd/popper.min.js"
})
app.scripts.append_script({
    "external_url": "https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js"
})

AttributeError: type object 'JupyterDash' has no attribute 'css'

In [ ]:
# define app layout
app.layout = html.Div([
    html.Div([
        html.Div(
            [
                html.H1("Music Dashboard", className="text-center", id="heading")
            ], className="col-md-12"
        ),
    ], className="row"),

    html.Div(
        [  # dropdown and score
            html.Div([
                html.Div(
                    [
                        dcc.Dropdown(
                            options=[
                                {'label': 'userscore', 'value': 'userscore'},
                                {'label': 'metascore', 'value': 'metascore'},
                            ],
                            id='score-dropdown'
                        )
                    ], className="col-md-12"),
                html.Div(
                    html.Table(id='datatable', className="table col-md-12")
                ),
            ], className="col-md-6"),

            html.Div(
                [  # Line Chart
                    dcc.Graph(id='line-graph',
                              figure=go.Figure(
                                  data=[
                                      go.Scatter(
                                          x=df_linechart.year,
                                          y=df_linechart.userscore,
                                          mode='lines',
                                          name='user score'
                                      ),
                                      go.Scatter(
                                          x=df_linechart.year,
                                          y=df_linechart.metascore,
                                          mode='lines',
                                          name='meta score'
                                      ),
                                  ],
                                  layout=go.Layout(title="Score trends")
                              )
                              ),
                ], className="col-md-6"
            ),
        ], className="row"),

    html.Div(
        [
            html.Div(
                [
                    dcc.Graph(id='bubble-chart',
                              figure=go.Figure(
                                  data=[
                                      go.Scatter(
                                          x=df_bubble.userscore,
                                          y=df_bubble.metascore,
                                          mode='markers',
                                          text=df_bubble.genre,

                                          marker=dict(
                                              color=random.sample(range(1, 200), 15),
                                              size=df_bubble.year,
                                              sizemode='area',
                                              sizeref=2. * max(df_bubble.year) / (40. ** 2),
                                              sizemin=4
                                          )
                                      )
                                  ],
                                  layout=go.Layout(title="Genre popularity")
                              )

                              )
                ], className="col-md-6"
            ),
            html.Div(
                [
                    dcc.Graph(id='bar-chart',
                              style={'margin-top': '20'})
                ], className="col-md-6"
            ),

        ], className="row"),

], className="container-fluid")

In [ ]:
##############################################################
# DATA CONTROL (CONTROLLER)
##############################################################
@app.callback(
    Output(component_id='datatable', component_property='children'),
    [Input(component_id='score-dropdown', component_property='value')]
)
def update_table(input_value):
    return generate_table(df_table.sort_values([input_value], ascending=[False]).reset_index())


@app.callback(
    Output(component_id='bar-chart', component_property='figure'),
    [Input(component_id='bubble-chart', component_property='hoverData')]
)
def update_graph(hoverData):
    return bar(hoverData)

In [ ]:
# Run app and display result inline in the notebook
app.run_server(mode='inline')